<a href="https://colab.research.google.com/github/kzon94/eve-marketer-v2/blob/main/eve_marketer_v2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧙‍♂️ Jedi

Before you can summon the full power of this notebook, you must first train in the ways of the **Jedi** (the Python one, not the lightsaber kind... unfortunately). Run the following command to install the dependency required:

In [ ]:
!pip install jedi

## 📄 Instructions

To run this notebook, make sure to upload the file **`materials_id_classified.csv`**, which is available in the project's GitHub repository. This file contains the list of mining materials, along with their **type IDs**, **classification** (*Ore, Gas, Ice*, etc.), and **tier** (based on their abundance and security level zone).

The script will query the **EVE Online ESI API** to fetch market data for each item in **Jita 4-4**, including:

- ✅ Current **buy** and **sell** prices  
- 📈 Historical **max buy prices** over the last **7** and **31** days  
- 📉 **Percentage change** in max buy price compared to current buy price  

The final result will be saved as a CSV file called **`mining_market_jita_v3.csv`**, making it compatible with **Excel** and other spreadsheet tools.


In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timezone, timedelta
from concurrent.futures import ThreadPoolExecutor
from google.colab import files

INPUT_FILE = "materials_id_classified.csv"
OUTPUT_FILE = "mining_market_jita_v3.csv"

HEADERS = {
    "User-Agent": "ElTiburonCazon/2.0 (contacto@example.com)"
}

REGION_ID = 10000002  # The Forge (Jita)
MARKET_HISTORY_ENDPOINT = "https://esi.evetech.net/latest/markets/{region_id}/history/?type_id={type_id}"
MARKET_ORDERS_ENDPOINT = "https://esi.evetech.net/latest/markets/{region_id}/orders/"
JITA_44_LOCATION_ID = 60003760

today = datetime.now(timezone.utc).date()
dates_filter = {
    "7d": today - timedelta(days=7),
    "31d": today - timedelta(days=31)
}

def fetch_buy_price_history(session, type_id):
    url = MARKET_HISTORY_ENDPOINT.format(region_id=REGION_ID, type_id=type_id)
    try:
        resp = session.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        df = pd.DataFrame(resp.json())
        if df.empty:
            return {f"max_buy_price_{k}": 0 for k in dates_filter}
        df['date'] = pd.to_datetime(df['date']).dt.date
        return {
            f"max_buy_price_{k}": df[df['date'] >= v]['highest'].max() if not df[df['date'] >= v].empty else 0
            for k, v in dates_filter.items()
        }
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch history for {type_id}: {e}")
        return {f"max_buy_price_{k}": 0 for k in dates_filter}

def fetch_current_prices(session, type_id):
    params = {"datasource": "tranquility", "type_id": type_id}
    try:
        resp = session.get(MARKET_ORDERS_ENDPOINT.format(region_id=REGION_ID), headers=HEADERS, params=params, timeout=10)
        resp.raise_for_status()
        orders = resp.json()
        buy_prices = [o['price'] for o in orders if o.get('is_buy_order') and o.get('location_id') == JITA_44_LOCATION_ID]
        sell_prices = [o['price'] for o in orders if not o.get('is_buy_order') and o.get('location_id') == JITA_44_LOCATION_ID]
        return max(buy_prices) if buy_prices else 0, min(sell_prices) if sell_prices else 0
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch current prices for {type_id}: {e}")
        return 0, 0

def process_row(session, row):
    tid, name, tier, material_type = row['type_id'], row['name'], row['tier'], row['material_type']
    current_buy, current_sell = fetch_current_prices(session, tid)
    history = fetch_buy_price_history(session, tid)
    return {
        "tier": tier, "name": name, "type_id": tid, "material_type": material_type,
        "current_buy_price": current_buy, "current_sell_price": current_sell, **history
    }

def calculate_percentage_changes(df):
    for period in ['7d', '31d']:
        df[f'change_max_buy_price_{period}'] = df.apply(
            lambda row: ((row['current_buy_price'] - row[f'max_buy_price_{period}']) / row[f'max_buy_price_{period}'] * 100)
            if row[f'max_buy_price_{period}'] != 0 else 0,
            axis=1
        )
    return df

def main():
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"File {INPUT_FILE} not found. Please upload it to Colab.")
        return

    required_cols = {'tier', 'name', 'type_id', 'material_type'}
    if not required_cols.issubset(df.columns):
        print("Missing required columns: 'tier', 'name', 'type_id', 'material_type'")
        return

    results = []
    with requests.Session() as session:
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(process_row, session, row) for _, row in df.iterrows()]
            for future in futures:
                result = future.result()
                if result:
                    results.append(result)
                time.sleep(0.1)

    results_df = pd.DataFrame(results)
    results_df = calculate_percentage_changes(results_df)

    results_df.to_csv(OUTPUT_FILE, index=False, sep=';', decimal=',')
    print(f"File saved: {OUTPUT_FILE}")
    files.download(OUTPUT_FILE)

if __name__ == "__main__":
    main()

## 📊 Market Graph Generator

- Be sure the file **`mining_market_jita_v3.csv`** has been generated in your Colab session.
- Select a **Tier** and **Material Type** from the dropdown menus to generate:

  - A **horizontal bar chart** showing **current buy prices**, sorted from lowest to highest.
  - A **logarithmic line chart** showing **price evolution** over time (**31d**, **7d**, and **Now**).

Both charts will be displayed in the notebook and **automatically downloaded** as a single PNG image.


In [ ]:
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files

INPUT_FILE = "mining_market_jita_v3.csv"

def load_data():
    try:
        df = pd.read_csv(INPUT_FILE, sep=';', decimal=',')
        return df
    except FileNotFoundError:
        print(f"{INPUT_FILE} not found. Please upload it to Colab.")
        return None

def generate_combined_plots(df, tier, material_type):
    filtered_df = df[(df['tier'] == tier) & (df['material_type'] == material_type)]
    if filtered_df.empty:
        print("No data available for the selected combination.")
        return

    # Bar chart
    filtered_df = filtered_df.sort_values(by='current_buy_price', ascending=True)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 16))

    y_labels = filtered_df['name']
    x_buy = filtered_df['current_buy_price']
    y_positions = range(len(filtered_df))

    bars = ax1.barh(y_positions, x_buy, height=0.6, label='Buy Price', color='steelblue')
    ax1.set_yticks(y_positions)
    ax1.set_yticklabels(y_labels)
    ax1.set_xlabel('Price (ISK)')
    ax1.set_title(f'Buy Prices in Tier {tier} - {material_type}')
    ax1.legend()
    ax1.set_xlim([0, x_buy.max() * 1.05])

    for bar in bars:
        label_x = bar.get_width() - (bar.get_width() * 0.1)
        ax1.text(label_x, bar.get_y() + bar.get_height() / 2, f'{bar.get_width():,.0f}', va='center', ha='right', color='white', fontsize=9)

    # Price evolution chart
    ax2.set_yscale('log')
    for _, row in filtered_df.iterrows():
        prices = [
            row['max_buy_price_31d'],
            row['max_buy_price_7d'],
            row['current_buy_price']
        ]
        if pd.isnull(prices).any():
            continue
        ax2.plot(['31d', '7d', 'Now'], prices, marker='o', label=row['name'])
        ax2.text(2, prices[2], f'{prices[2]:,.0f}', ha='left', va='bottom', fontsize=8)

    ax2.set_title(f'Price Evolution - Tier {tier} - {material_type}')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Buy Price (ISK)')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small')

    plt.tight_layout()
    filename = f"{tier}_{material_type}_combined_charts.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    files.download(filename)
    plt.show()

def main_menu(df):
    tier_dropdown = widgets.Dropdown(
        options=['-Tier-'] + sorted(df['tier'].dropna().unique()),
        description='Tier:',
        value='-Tier-'
    )

    material_type_dropdown = widgets.Dropdown(
        options=['-Type-'],
        description='Material:',
        value='-Type-'
    )

    def update_material_types(*args):
        selected_tier = tier_dropdown.value
        if selected_tier != '-Tier-':
            types = df[df['tier'] == selected_tier]['material_type'].dropna().unique()
            material_type_dropdown.options = ['-Type-'] + sorted(types)
        else:
            material_type_dropdown.options = ['-Type-']

    tier_dropdown.observe(update_material_types, names='value')

    def on_graph_button_click(b):
        if tier_dropdown.value != '-Tier-' and material_type_dropdown.value != '-Type-':
            generate_combined_plots(df, tier_dropdown.value, material_type_dropdown.value)
        else:
            print("Please select both a Tier and a Material Type.")

    graph_button = widgets.Button(description="Generate Graphs")
    graph_button.on_click(on_graph_button_click)

    display(widgets.VBox([tier_dropdown, material_type_dropdown, graph_button]))

df = load_data()
if df is not None:
    main_menu(df)